In [1]:
import json
import os
import time
import random
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import shutil
import collections

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets
from torchvision import transforms
from torchvision import models
from torchsummary import summary

from model_pytorch import EfficientNet
from utils import Bar,Logger, AverageMeter, accuracy, mkdir_p, savefig
from warmup_scheduler import GradualWarmupScheduler
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split

from scipy.ndimage.filters import gaussian_filter
import cv2

from PIL import ImageFile, ImageOps
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [2]:
# GPU Device
gpu_id = 3
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu_id)
use_cuda = torch.cuda.is_available()
print("GPU device %d:" %(gpu_id), use_cuda)

GPU device 3: True


# Arguments

In [3]:
source_dir = '/media/data2/dataset/GAN_ImageData/StarGAN_128/'
target_dir = '/media/data2/dataset/GAN_ImageData/PGGAN_128/'

In [4]:
pretrained = './log/star/128/b0/aug/checkpoint.pth.tar'
resume = ''

In [5]:
# Model
model_name = 'efficientnet-b0' # b0-b7 scale

# Optimization
num_classes = 2
epochs = 1000
start_epoch = 0
train_batch = 250
test_batch = 250
lr = 0.01
schedule = [50, 250, 500, 750]
momentum = 0.1
gamma = 0.1 # LR is multiplied by gamma on schedule

# CheckPoint
checkpoint = './log/star/128/b0/to_pggan/2000shot/general' # dir
if not os.path.isdir(checkpoint):
    os.makedirs(checkpoint)
num_workers = 8

# Seed
manual_seed = 7
random.seed(manual_seed)
torch.cuda.manual_seed_all(manual_seed)

# Image
size = (128, 128)

# cutmix
cm_prob = 0.2
cm_beta = 1.0

# augmentation
blur_prob = 0.2
blog_sig = 0.5
jpg_prob = 0.2

best_acc = 0

In [6]:
state = {}
state['num_classes'] = num_classes
state['epochs'] = epochs
state['start_epoch'] = start_epoch
state['train_batch'] = train_batch
state['test_batch'] = test_batch
state['lr'] = lr
state['schedule'] = schedule
state['momentum'] = momentum
state['gamma'] = gamma

In [7]:
def data_augment(img):
    img = np.array(img)

    if random.random() < blur_prob:
        sig = np.random.uniform(0.0, 3.0)
        gaussian_blur(img, sig)

    if random.random() < jpg_prob:
        qual = np.random.uniform(30.0, 100.0)
        img = cv2_jpg(img, qual)

    return Image.fromarray(img)


def gaussian_blur(img, sigma):
    gaussian_filter(img[:,:,0], output=img[:,:,0], sigma=sigma)
    gaussian_filter(img[:,:,1], output=img[:,:,1], sigma=sigma)
    gaussian_filter(img[:,:,2], output=img[:,:,2], sigma=sigma)


def cv2_jpg(img, compress_val):
    img_cv2 = img[:,:,::-1]
    encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), compress_val]
    result, encimg = cv2.imencode('.jpg', img_cv2, encode_param)
    decimg = cv2.imdecode(encimg, 1)
    return decimg[:,:,::-1]

In [8]:
def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

# Dataset

In [9]:
train_dir = os.path.join(source_dir, 'pggan/2000_shot_only')
val_target_dir = os.path.join(target_dir, 'validation')
val_source_dir = os.path.join(source_dir, 'validation')

train_aug = transforms.Compose([
    transforms.Lambda(lambda img: data_augment(img)),
    transforms.Resize(size),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
#     transforms.RandomErasing(p=0.3, scale=(0.02, 0.10), ratio=(0.3, 3.3), value=0, inplace=True),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
val_aug = transforms.Compose([
    transforms.Resize(size),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# pin_memory : cuda pin memeory use
train_loader = DataLoader(datasets.ImageFolder(train_dir, transform=train_aug),
                          batch_size=train_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_target_loader = DataLoader(datasets.ImageFolder(val_target_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_source_loader = DataLoader(datasets.ImageFolder(val_source_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)

# Model

In [10]:
student_model = EfficientNet.from_name(model_name, num_classes=num_classes,
                              override_params={'dropout_rate':0.2, 'drop_connect_rate':0.2})

# Pre-trained
if pretrained:
    print("=> using pre-trained model '{}'".format(pretrained))
    student_model.load_state_dict(torch.load(pretrained)['state_dict'])

=> using pre-trained model './log/star/128/b0/aug/checkpoint.pth.tar'


In [11]:
student_model.to('cuda')
cudnn.benchmark = True
print('    Total params: %.2fM' % (sum(p.numel() for p in student_model.parameters())/1000000.0))

    Total params: 4.01M


In [12]:
for idx, param in enumerate(student_model.parameters()):
    param.reqiures_grad = False
    if idx == 182:
        break

In [13]:
criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.SGD(student_model.parameters(), lr=lr, momentum=momentum)
# optimizer = optim.Adam(student_model.parameters())
scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)
scheduler_warmup = GradualWarmupScheduler(optimizer, multiplier=4, total_epoch=10, after_scheduler=scheduler_cosine)

# Loss

In [14]:
# Resume
if resume:
    print('==> Resuming from checkpoint..')
    checkpoint = os.path.dirname(resume)
#     checkpoint = torch.load(resume)
    resume = torch.load(resume)
    best_acc = resume['best_acc']
    start_epoch = resume['epoch']
    student_model.load_state_dict(resume['state_dict'])
    optimizer.load_state_dict(resume['optimizer'])
    logger = Logger(os.path.join(checkpoint, 'log.txt'), resume=True)
else:
    logger = Logger(os.path.join(checkpoint, 'log.txt'))
    logger.set_names(['Learning Rate', 'Train Loss', 'Valid Loss', 'Source Loss', 'Train Acc.', 'Valid Acc.', 'Source ACC.', 'Train AUROC', 'Valid AUROC', 'Source AUROC'])

# Train

In [15]:
def train(train_loader, model, criterion, optimizer, epoch, use_cuda):
    model.train()
    
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    arc = AverageMeter()
    end = time.time()
    cls_losses = AverageMeter()
    sp_losses = AverageMeter()
    main_losses = AverageMeter()
    
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        batch_size = inputs.size(0)
        if batch_size < train_batch:
            continue
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
            
        r = np.random.rand(1)
        if cm_beta > 0 and r < cm_prob:
            
            rand_index = torch.randperm(inputs.size()[0]).cuda()
            tt= targets[rand_index]
            boolean = targets==tt
            rand_index = rand_index[boolean]
            lam = np.random.beta(cm_beta, cm_beta)
            bbx1, bby1, bbx2, bby2 = rand_bbox(inputs.size(), lam)
            inputs[boolean, :, bbx1:bbx2, bby1:bby2] = inputs[rand_index, :, bbx1:bbx2, bby1:bby2]

        # compute output
        outputs = model(inputs)
        loss_main = criterion(outputs, targets)
        loss =  loss_main
        loss_cls = 0
        loss_sp = 0

        # measure accuracy and record loss
        prec1 = accuracy(outputs.data, targets.data)
#         auroc = roc_auc_score(targets.cpu().detach().numpy(), outputs.cpu().detach().numpy()[:,1])
        losses.update(loss.data.tolist(), inputs.size(0))
        top1.update(prec1[0], inputs.size(0))
#         arc.update(auroc, inputs.size(0))
        cls_losses.update(loss_cls, inputs.size(0))
        sp_losses.update(loss_sp, inputs.size(0))
        main_losses.update(loss_main.tolist(), inputs.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        
#         if batch_idx % 100 == 0:
#             print('{batch}/{size} | Loss:{loss:.4f} | MainLoss:{main:.4f} | SPLoss:{sp:.4f} | CLSLoss:{cls:.4f} | top1:{tp1:.4f} | AUROC:{ac:.4f}'.format(
#                      batch=batch_idx+1, size=len(train_loader), loss=losses.avg, main=main_losses.avg, sp=sp_losses.avg, cls=cls_losses.avg, tp1=top1.avg, ac=arc.avg))
    print('Train | {batch}/{size} | Loss:{loss:.4f} | MainLoss:{main:.4f} | SPLoss:{sp:.4f} | CLSLoss:{cls:.4f} | top1:{tp1:.4f} | AUROC:{ac:.4f}'.format(
                     batch=batch_idx+1, size=len(train_loader), loss=losses.avg, main=main_losses.avg, sp=sp_losses.avg, cls=cls_losses.avg, tp1=top1.avg, ac=arc.avg))
    return (losses.avg, top1.avg, arc.avg)

In [16]:
def test(val_loader, model, criterion, epoch, use_cuda):
    global best_acc

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    arc = AverageMeter()
    cls_losses = AverageMeter()
    sp_losses = AverageMeter()
    main_losses = AverageMeter()


    # switch to evaluate mode
    model.eval()

    end = time.time()
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(val_loader):
            # measure data loading time
            data_time.update(time.time() - end)

            if use_cuda:
                inputs, targets = inputs.cuda(), targets.cuda()

            # compute output
            outputs = model(inputs)
            loss_main = criterion(outputs, targets)
            loss = loss_main
            loss_cls = 0
            loss_sp = 0

            # measure accuracy and record loss
            prec1 = accuracy(outputs.data, targets.data)
            auroc = roc_auc_score(targets.cpu().detach().numpy(), outputs.cpu().detach().numpy()[:,1])
            losses.update(loss.data.tolist(), inputs.size(0))
            top1.update(prec1[0], inputs.size(0))
            arc.update(auroc, inputs.size(0))
            cls_losses.update(loss_cls, inputs.size(0))
            sp_losses.update(loss_sp, inputs.size(0))
            main_losses.update(loss_main.tolist(), inputs.size(0))


            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

    print('Test | {batch}/{size} | Loss:{loss:.4f} | MainLoss:{main:.4f} | SPLoss:{sp:.4f} | CLSLoss:{cls:.4f} | top1:{tp1:.4f} | AUROC:{ac:.4f}'.format(
                     batch=batch_idx+1, size=len(train_loader), loss=losses.avg, main=main_losses.avg, sp=sp_losses.avg, cls=cls_losses.avg, tp1=top1.avg, ac=arc.avg))
    return (losses.avg, top1.avg, arc.avg)

In [17]:
def save_checkpoint(state, is_best, checkpoint='checkpoint', filename='checkpoint.pth.tar'):
    filepath = os.path.join(checkpoint, filename)
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(checkpoint, 'model_best.pth.tar'))

def adjust_learning_rate(optimizer, epoch):
    global state
    lr_set = [1, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5]
    lr_list = schedule.copy()
    lr_list.append(epoch)
    lr_list.sort()
    idx = lr_list.index(epoch)
    state['lr'] *= lr_set[idx]
    for param_group in optimizer.param_groups:
        param_group['lr'] = state['lr']

In [ ]:
for epoch in range(start_epoch, epochs):
    state['lr'] = optimizer.state_dict()['param_groups'][0]['lr']
    adjust_learning_rate(optimizer, epoch)
    print('\nEpoch: [%d | %d] LR: %f' % (epoch + 1, epochs, state['lr']))
    
    train_loss, train_acc, train_auroc = train(train_loader, student_model, criterion, optimizer, epoch, use_cuda)
    test_loss, test_acc, test_auroc = test(val_target_loader, student_model, criterion, epoch, use_cuda)
    source_loss, source_acc, source_auroc = test(val_source_loader, student_model, criterion, epoch, use_cuda)


    logger.append([state['lr'], train_loss, test_loss,  source_loss, train_acc, test_acc,source_acc, train_auroc, test_auroc, source_auroc])
    is_best = test_acc > best_acc
    best_acc = max(test_acc, best_acc)
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict' : student_model.state_dict(),
        'acc': test_acc,
        'best_acc': best_acc,
        'optimizer': optimizer.state_dict(),
    }, is_best, checkpoint=checkpoint)
    scheduler_warmup.step()


Epoch: [1 | 1000] LR: 0.010000
Train | 16/16 | Loss:0.7799 | MainLoss:0.7799 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:67.5500 | AUROC:0.0000
Test | 129/16 | Loss:0.4770 | MainLoss:0.4770 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:77.1838 | AUROC:0.8568
Test | 123/16 | Loss:0.3200 | MainLoss:0.3200 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:86.6186 | AUROC:0.9977

Epoch: [2 | 1000] LR: 0.013000
Train | 16/16 | Loss:0.5226 | MainLoss:0.5226 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:74.8000 | AUROC:0.0000
Test | 129/16 | Loss:0.3945 | MainLoss:0.3945 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:81.8972 | AUROC:0.9214
Test | 123/16 | Loss:0.2688 | MainLoss:0.2688 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:88.3093 | AUROC:0.9974

Epoch: [3 | 1000] LR: 0.016000
Train | 16/16 | Loss:0.4710 | MainLoss:0.4710 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:78.1250 | AUROC:0.0000
Test | 129/16 | Loss:0.3067 | MainLoss:0.3067 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:87.8411 | AUROC:0.9576
Test | 123/16 | Loss:0.2994 | Ma

Train | 16/16 | Loss:0.2618 | MainLoss:0.2618 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:88.1250 | AUROC:0.0000
Test | 129/16 | Loss:0.1751 | MainLoss:0.1751 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:93.2025 | AUROC:0.9985
Test | 123/16 | Loss:0.1535 | MainLoss:0.1535 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:93.8139 | AUROC:0.9911

Epoch: [25 | 1000] LR: 0.039983
Train | 16/16 | Loss:0.2585 | MainLoss:0.2585 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:88.1250 | AUROC:0.0000
Test | 129/16 | Loss:0.1834 | MainLoss:0.1834 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:92.7632 | AUROC:0.9987
Test | 123/16 | Loss:0.1509 | MainLoss:0.1509 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:94.0760 | AUROC:0.9896

Epoch: [26 | 1000] LR: 0.039981
Train | 16/16 | Loss:0.2614 | MainLoss:0.2614 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:88.2000 | AUROC:0.0000
Test | 129/16 | Loss:0.1007 | MainLoss:0.1007 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:96.8162 | AUROC:0.9986
Test | 123/16 | Loss:0.2131 | MainLoss:0.2131 | SPLoss:0.0000 

Train | 16/16 | Loss:0.1993 | MainLoss:0.1993 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:91.5500 | AUROC:0.0000
Test | 129/16 | Loss:0.1789 | MainLoss:0.1789 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:93.1090 | AUROC:0.9981
Test | 123/16 | Loss:0.1800 | MainLoss:0.1800 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:92.5918 | AUROC:0.9809

Epoch: [48 | 1000] LR: 0.039872
Train | 16/16 | Loss:0.1855 | MainLoss:0.1855 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:92.0750 | AUROC:0.0000
Test | 129/16 | Loss:0.1040 | MainLoss:0.1040 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:96.0810 | AUROC:0.9984
Test | 123/16 | Loss:0.2197 | MainLoss:0.2197 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:91.0059 | AUROC:0.9819

Epoch: [49 | 1000] LR: 0.039865
Train | 16/16 | Loss:0.1817 | MainLoss:0.1817 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:92.7500 | AUROC:0.0000
Test | 129/16 | Loss:0.1250 | MainLoss:0.1250 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:95.3707 | AUROC:0.9986
Test | 123/16 | Loss:0.2115 | MainLoss:0.2115 | SPLoss:0.0000 

Train | 16/16 | Loss:0.1555 | MainLoss:0.1555 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:93.9750 | AUROC:0.0000
Test | 129/16 | Loss:0.1412 | MainLoss:0.1412 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:94.6885 | AUROC:0.9980
Test | 123/16 | Loss:0.2086 | MainLoss:0.2086 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:91.7530 | AUROC:0.9812

Epoch: [71 | 1000] LR: 0.003966
Train | 16/16 | Loss:0.1505 | MainLoss:0.1505 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:94.1000 | AUROC:0.0000
Test | 129/16 | Loss:0.1459 | MainLoss:0.1459 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:94.5109 | AUROC:0.9979
Test | 123/16 | Loss:0.2063 | MainLoss:0.2063 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:91.8480 | AUROC:0.9812

Epoch: [72 | 1000] LR: 0.003965
Train | 16/16 | Loss:0.1476 | MainLoss:0.1476 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:94.0250 | AUROC:0.0000
Test | 129/16 | Loss:0.1404 | MainLoss:0.1404 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:94.7321 | AUROC:0.9980
Test | 123/16 | Loss:0.2097 | MainLoss:0.2097 | SPLoss:0.0000 

Train | 16/16 | Loss:0.1382 | MainLoss:0.1382 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:94.4750 | AUROC:0.0000
Test | 129/16 | Loss:0.1411 | MainLoss:0.1411 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:94.8006 | AUROC:0.9979
Test | 123/16 | Loss:0.2166 | MainLoss:0.2166 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:91.6284 | AUROC:0.9806

Epoch: [94 | 1000] LR: 0.003934
Train | 16/16 | Loss:0.1449 | MainLoss:0.1449 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:94.0250 | AUROC:0.0000
Test | 129/16 | Loss:0.1462 | MainLoss:0.1462 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:94.5701 | AUROC:0.9978
Test | 123/16 | Loss:0.2136 | MainLoss:0.2136 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:91.7267 | AUROC:0.9804

Epoch: [95 | 1000] LR: 0.003932
Train | 16/16 | Loss:0.1502 | MainLoss:0.1502 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:93.7500 | AUROC:0.0000
Test | 129/16 | Loss:0.1309 | MainLoss:0.1309 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:95.2056 | AUROC:0.9980
Test | 123/16 | Loss:0.2236 | MainLoss:0.2236 | SPLoss:0.0000 